# Análisis exploratorio del los anuncios de ventas de coches 

Trabajo como analista de datos para una compañia dedicada a promocionar anuncios para ventas de coches de diferentes marcas y diferentes clases de coches. A partir del conjunto de datos dado, se realizarán estudios y gráficas sobre las diferentes marcas y clases de coches con el fin de utilizar dicha información para crear una aplicación web interactiva que permita a los usuarios revisar los resultados de nuestro estudio.


# 1. Inicialización

In [1]:
# Cargar todas las librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
import plotly.express as px

## 1.2 Cargar datos

In [2]:
# Carga los archivos de datos del DataFrame
car_data = pd.read_csv(r'C:\Users\William Guacaneme\Data Analyst TT\Sprint 7\sprint-7-project\vehicles_us.csv') # leer los datos

# 2 Revisión de datos

In [3]:
# Imprime la información general/resumida sobre el DataFrame de los juegos
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB


In [4]:
# Imprime una muestra de los datos
car_data.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17


In [5]:
# Búsqueda valores ausentes
print(car_data.isna().sum())

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64


In [6]:
# Búsqueda de duplicados
print(car_data.duplicated().sum())

0


In [7]:
# Detección de duplicados implícitos
print(car_data['model'].nunique())

100


In [8]:
# Filtrado por modelo y año
car_data.groupby(['model', 'model_year'])['price'].count()

model              model_year
acura tl           1999.0         3
                   2001.0         6
                   2002.0         2
                   2003.0        10
                   2004.0        16
                                 ..
volkswagen passat  2014.0        44
                   2015.0        26
                   2016.0        10
                   2017.0        10
                   2018.0         1
Name: price, Length: 2226, dtype: int64

**OBSERVACIONES**

- El DataFrame tiene 51525 entradas y 13 columnas.
- Hay valores ausentes en el DataFrame que más adelante determinaremos que hacer con ellos.
- No hay filas completamente duplicadas, pero realizando la búsqueda de duplicados implícitos, vemos que en la columna 'model' hay valores duplicados debido a que pueden existir el mismo modelo pero de un año difertente; pasa lo mismo ya que si se tiene un modelo específico y un año del modelo específico pueden variar sus precios.
- Hay varias columnas a las que se le deben convertir a tipo de datos necesario, tales como: `model_year`, `cylinders` y `odometer` de float64 a int64 y `date_posted` de object a datetime.
- Se puede agregar una columna con el nombre del fabricante para obtener unas gráficas mucho más generales.

## 2.1 Corregir datos

### 2.1.1 Valores ausentes

En este caso los valores ausentes pueden ser causados por diferentes razones, tales como: problemas en la fuente de datos, errores en la captura de datos, incompatibilidad de formates, etc. Estos afectan al análisis exploratorio porque cada dato representa una caracteristica que al final puede ser utilizada para mostrar en la aplicación siendo una información útil para los usuarios que ven los anuncios de los coches en venta. No es el objetivo en este proyecto de hacer todo el proceso correcto de limpieza de datos, pero se procederá a hacer lo siguiente:

- Principalmente, se decide no eliminar ninguna fila que presente valores ausentes, ya que al eliminarse podría cambiar drásticamente la información final.

- La columna `model_year` y `cylindres` tienen 3619 y 5260 valores ausentes respectivamente. Para este análisis consideramos que el año del modelo y cilindros son características importantes que si no se reemplazan los valores NaN por algún valor numérico se pierde información valiosa al momento de generar gráficas, por lo que se reemplazaran los valores ausentes por valores int 0, como indicador de que del vehículo no se conoce su año del modelo y el número de cilindros.

- IMPORTANTE: A pesar que a continuacion se realiza el llenado de datos se decide no activar el código ya que modifica el aspecto de las gráficas que se quieren mostra mas adelante.


In [21]:
# completar valores ausentes de las columnas `model_year` y `cylinders`
# car_data['model_year'] = car_data['model_year'].fillna(0)
# car_data['cylinders'] = car_data['cylinders'].fillna(0)

- Para la columna `odometer` se decide no realizar nada con los valores ausentes, ya que no tiene sentido ponerle 0 ya que se puede entender con que el vehículo no tiene kilometros recorridos.

- Las columnas `paint_color` y `is_4wd` son irrelevantes para éste análisis o para lo que se quiere utilizar de información para mostrar en la aplicación, por lo que no se realizará nada con los valores ausentes.

### 2.1.2 Convertir datos en los tipos necesarios

In [ ]:
# Convierte los datos en los tipos necesarios. 
# car_data['model_year'] = car_data['model_year'].astype(int)
# car_data['cylinders'] = car_data['cylinders'].astype(int)

In [11]:
# Cambio a tipo de dato datetime64 
car_data['date_posted'] = pd.to_datetime(car_data['date_posted'], errors='coerce') 
# No es necesario cambiar el formato ya que Python lo detecta como %Y-%m-%d, creemos que esta en un formato correcto.

## 2.2 Enriquecer datos

In [12]:
# creación de la columna por fabricante
car_data['manufacturer'] = car_data['model'].str.split().str[0]

In [13]:
car_data.head(10)

,price,model_year,model,condition,cylinders,fuel,odometer,transmission,type,paint_color,is_4wd,date_posted,days_listed,manufacturer
0,9400,2011.0,bmw x5,good,6.0,gas,145000.0,automatic,SUV,NaN,1.0,2018-06-23,19,bmw
1,25500,NaN,ford f-150,good,6.0,gas,88705.0,automatic,pickup,white,1.0,2018-10-19,50,ford
2,5500,2013.0,hyundai sonata,like new,4.0,gas,110000.0,automatic,sedan,red,NaN,2019-02-07,79,hyundai
3,1500,2003.0,ford f-150,fair,8.0,gas,NaN,automatic,pickup,NaN,NaN,2019-03-22,9,ford
4,14900,2017.0,chrysler 200,excellent,4.0,gas,80903.0,automatic,sedan,black,NaN,2019-04-02,28,chrysler
5,14990,2014.0,chrysler 300,excellent,6.0,gas,57954.0,automatic,sedan,black,1.0,2018-06-20,15,chrysler
6,12990,2015.0,toyota camry,excellent,4.0,gas,79212.0,automatic,sedan,white,NaN,2018-12-27,73,toyota
7,15990,2013.0,honda pilot,excellent,6.0,gas,109473.0,automatic,SUV,black,1.0,2019-01-07,68,honda
8,11500,2012.0,kia sorento,excellent,4.0,gas,104174.0,automatic,SUV,NaN,1.0,2018-07-16,19,kia
9,9200,2008.0,honda pilot,excellent,NaN,gas,147191.0,automatic,SUV,blue,1.0,2019-02-15,17,honda


In [14]:
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   price         51525 non-null  int64         
 1   model_year    47906 non-null  float64       
 2   model         51525 non-null  object        
 3   condition     51525 non-null  object        
 4   cylinders     46265 non-null  float64       
 5   fuel          51525 non-null  object        
 6   odometer      43633 non-null  float64       
 7   transmission  51525 non-null  object        
 8   type          51525 non-null  object        
 9   paint_color   42258 non-null  object        
 10  is_4wd        25572 non-null  float64       
 11  date_posted   51525 non-null  datetime64[ns]
 12  days_listed   51525 non-null  int64         
 13  manufacturer  51525 non-null  object        
dtypes: datetime64[ns](1), float64(4), int64(2), object(7)
memory usage: 5.5+ MB


# 3. Analizar datos

## 3.1 Tipos de vehículos por fabricante

In [24]:
fig_1 = px.bar(car_data, x="manufacturer", color="type")
fig_1.show()

## 3.2 Modelos de vehículos por fabricante

In [25]:
fig_2 = px.bar(car_data, x="model", color="manufacturer")
fig_2.show()

## 3.3 Histograma de condición vs año del modelo

In [26]:
fig_3 = px.histogram(car_data, x='model_year', color='condition', nbins=50, title=' Hiostograma de condición vs año del modelo')
fig_3.show()

## 3.4 Comparar distribución de precio entre fabricantes

In [41]:
fig_4 = px.histogram(car_data, x="price", color="manufacturer")
fig_4.show()